In [ ]:
## Text Generation with GPT2 Model using HuggingFace

In [1]:
!pip install transformers

we have many techniques to generate text data,but we are using only techniques

1.Beam search decoding
2.Nucleus search decoding

these are the two most famous techniques so we will be using

AutoTokenizer--- if you provide a checkpoint model then it will automatically fetch a corresponding tokenizer from the internet or from hugging face.
        
        or

AutoTokenizer is a class in the Hugging Face Transformers library that automatically selects and loads the right tokenizer for a pre-trained model

AutoModelForCausalLM--- this model is used to generate text data.LM means Language Model.

In [2]:
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
model_name = "gpt2-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name).to(device) # by default this model is downloaded on cpu but once it is downloaded we need to shift it to our gpu for a faster inference,AutoModelForCausalLM is used for text generation,

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [6]:
input_text = "i am really impressed"
max_length = 128 # 128 words i want to generate

input_ids = tokenizer(input_text,return_tensors="pt") # before data goes your transformers encoder and decoder your data is tokenized before that and then embedding it generated so we need to do tokenization before we feed our data to our model

In [7]:
input_ids

{'input_ids': tensor([[   72,   716,  1107, 12617]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [8]:
input_ids = input_ids['input_ids'].to(device) # we need to also shift our tokenizer to gpu, so that it can performs faster, so input id goes to gpu,now your model is ready to generate the text data

In [9]:
output=model.generate(input_ids,max_length=max_length,num_beams=5,do_sample=False) # num_beams=5 number of branches,,,,do_sample=False--its going to use here beam search without any top case sampling

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
output # convert encoded data into text data

tensor([[   72,   716,  1107, 12617,   351,   262,  3081,   286,   262,  1720,
           290,   262,  6491,  2139,    13,   198,   198, 15322,   642,   503,
           286,   642,   416, 19200,   422,  3878,  1720,     0,   314,   423,
           587,  1262,   428,  1720,   329,   625,   257,   614,   783,   290,
           314,   423,  1239,   550,   257,  1917,   351,   340,    13,   314,
           423,   587,  1262,   428,  1720,   329,   625,   257,   614,   783,
           290,   314,   423,  1239,   550,   257,  1917,   351,   340,    13,
           314,   423,   587,  1262,   428,  1720,   329,   625,   257,   614,
           783,   290,   314,   423,  1239,   550,   257,  1917,   351,   340,
            13,   314,   423,   587,  1262,   428,  1720,   329,   625,   257,
           614,   783,   290,   314,   423,  1239,   550,   257,  1917,   351,
           340,    13,   314,   423,   587,  1262,   428,  1720,   329,   625,
           257,   614,   783,   290,   314,   423,  

In [13]:
print(tokenizer.decode(output[0])) # senetence repetation is occur to avoid this we need to introduce no_repeat_ngram_size=2

i am really impressed with the quality of the product and the customer service.

Rated 5 out of 5 by Anonymous from Great product! I have been using this product for over a year now and I have never had a problem with it. I have been using this product for over a year now and I have never had a problem with it. I have been using this product for over a year now and I have never had a problem with it. I have been using this product for over a year now and I have never had a problem with it. I have been using this product for over a year now and I have never had


In [14]:
output=model.generate(input_ids,max_length=max_length,num_beams=5,do_sample=False,no_repeat_ngram_size=2)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i am really impressed with the quality of the product and the customer service.

Rated 5 out of 5 by Anonymous from Great product! I have been using this product for over a year now and I am very pleased with it. It is very easy to use and works great. I would recommend it to anyone who is looking for a product that will last a long time.<|endoftext|>


Beam Search--- advantage

   -all the repetition is gone here.

 problem with the beam search is

   - if you run it multiple time,the probability is that you are going to see here almost same type of the text data every time.


## Nucleus Search

 we will be deciding there some probability based on that probability we
 will be selecting the samples.

 different text data is generated in every time

In [16]:
output=model.generate(input_ids,max_length=max_length,do_sample=True,top_p=0.9) # do_sample=True-- because this would be a sample method,,, this 0.9 is like the probability percentile from where these most probable tokens will be selected and thereafter a sampling will be done on that.
print(tokenizer.decode(output[0]))
# purpose of this top p=0.1==== we are talking here a huge amount of tokens for random sampling,text data would be quite low.
# if u select very high top p like o.9999,u will be selecting here a very small dictionary
# if u want a little bit more variance in your text data then you can lower your top p

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


i am really impressed, this is my second purchase from you guys and I have to say you have the best price tag in the whole world. If you don't have a laptop I suggest you buy one. It will be the best thing that ever happened to you. Your customer service is amazing and they will keep your purchase smooth. I only wish you guys are in the USA. They are my second and third home in Texas. Thanks again and have a great day.

Amazing. Very fast, friendly, knowledgeable staff. Super fast shipping as well.

I ordered it today. It was in two days and i'll
